In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/SNLP/Extra')

In [ ]:

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


from loaderglove import Lang, ToxicityDataset, normalizeString, collate, unicodeToAscii, load_glove_embeddings
from encoder import Encoder, TransformerClassifier

In [ ]:

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


from loaderglove import Lang, ToxicityDataset, normalizeString, collate, unicodeToAscii, load_glove_embeddings
from encoder import Encoder, TransformerClassifier

glove_file_path = '/content/drive/MyDrive/SNLP/Extra/glove.6B.100d.txt'


def compute_f1(model, val_loader):
    model.eval()
    f1 = []
    for i, data in enumerate(val_loader):
        inputs, mask, labels = data
        inputs = inputs.to(device)
        mask = mask.to(device)
        labels = labels.to(torch.float32).reshape(labels.size(0), 1).to(device)

        outputs = model(inputs, mask)

        out = torch.round(torch.sigmoid(outputs))
        tp = torch.sum(out * labels).item()
        fp = torch.sum(out * (1 - labels)).item()
        fn = torch.sum((1 - out) * labels).item()
        if tp == 0:
            f1.append(0)
        else:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1.append(2 * precision * recall / (precision + recall))

    f1 = np.mean(f1)
    return f1


def compute_accuracy(model, val_loader):
    model.eval()
    accuracy = []
    for i, data in enumerate(val_loader):
        inputs, mask, labels = data
        inputs = inputs.to(device)
        mask = mask.to(device)
        labels = labels.to(torch.float32).reshape(labels.size(0), 1).to(device)

        outputs = model(inputs, mask)

        out = torch.round(torch.sigmoid(outputs))
        accuracy.append(torch.sum(out == labels).item() / labels.size(0))

    accuracy = np.mean(accuracy)
    return accuracy

print(f'Loading data... \n')

skip_training = False

glove_embeddings = load_glove_embeddings(glove_file_path)
if 'UNK' not in glove_embeddings:
    glove_embeddings['UNK'] = np.random.randn(len(glove_embeddings['the']))
lang = Lang("eng", glove_embeddings)
data = pd.read_csv('/content/drive/My Drive/SNLP/Extra/data/train_2024.csv', quoting = 3)
df = pd.DataFrame(data)
for sentence in df['text']:
    lang.addSentence(normalizeString(unicodeToAscii(sentence)))

trainset = ToxicityDataset('/content/drive/MyDrive/SNLP/Extra/data/train_2024.csv', 'id', 'text', 'label', lang)
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=collate)

val_data = pd.read_csv('/content/drive/MyDrive/SNLP/Extra/data/dev_2024.csv', quoting = 3)
val_df = pd.DataFrame(val_data)

valset = ToxicityDataset('/content/drive/MyDrive/SNLP/Extra/data/dev_2024.csv', 'id', 'text', 'label', lang)
val_loader = DataLoader(valset, batch_size=64, shuffle=False, collate_fn=collate)

print(f'Vocabulary size: {lang.n_words}')
#Test if trainset contains correct number of rows
if len(trainset) == 99000:
    print("Trainset loaded correctly")
else:
    exit()

print("Trainset loaded \n")



Loading data... 

Vocabulary size: 70145
Trainset loaded correctly
Trainset loaded 



In [ ]:
embed_size = 256

embeddings_matrix = lang.get_embeddings_matrix()

np.shape(embeddings_matrix)

torch.Size([70145, 100])

In [ ]:
embed_size = 256

embeddings_matrix = lang.get_embeddings_matrix()

transformer_encoder = Encoder(src_vocab_size=trainset.lang.n_words, embedding_layer=nn.Embedding.from_pretrained(embeddings_matrix, freeze=False), n_blocks=2, n_features=embed_size, n_heads=4, n_hidden=512, dropout=0.1, max_length=5000)
encoder_classifier = TransformerClassifier(transformer_encoder, n_features=embed_size)

print("Model loaded \n")

model = encoder_classifier
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas = (0.9, 0.98), eps=1e-9)
criterion = nn.BCEWithLogitsLoss()

print(f'Entering the training loop \n')

if not skip_training:
    epochs = 25
    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        model.train()

        for i, data in enumerate(train_loader):
            inputs, mask, labels = data

            inputs = inputs.to(device)
            mask = mask.to(torch.float32).to(device)
            labels = labels.to(torch.float32).reshape(labels.size(0), 1).to(device)

            optimizer.zero_grad()

            outputs = model(inputs, mask)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()


            predicted = torch.round(torch.sigmoid(outputs))
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {loss.item():.4f}")

    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = correct / total * 100.0

print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


val_accuracy = compute_accuracy(model, val_loader)
print(f"Validation accuracy: {val_accuracy:.2f}%")
print(f"Validation F1: {compute_f1(model, val_loader):.2f}%")



torch.save(model.state_dict(), 'encoder.pth')

print(f'Model saved \n')


Streaming output truncated to the last 5000 lines.
Epoch 22, Batch 1193, Loss: 0.0352
Epoch 22, Batch 1194, Loss: 0.1099
Epoch 22, Batch 1195, Loss: 0.1121
Epoch 22, Batch 1196, Loss: 0.1503
Epoch 22, Batch 1197, Loss: 0.0766
Epoch 22, Batch 1198, Loss: 0.1797
Epoch 22, Batch 1199, Loss: 0.1876
Epoch 22, Batch 1200, Loss: 0.1433
Epoch 22, Batch 1201, Loss: 0.1428
Epoch 22, Batch 1202, Loss: 0.1114
Epoch 22, Batch 1203, Loss: 0.1840
Epoch 22, Batch 1204, Loss: 0.1619
Epoch 22, Batch 1205, Loss: 0.3134
Epoch 22, Batch 1206, Loss: 0.2082
Epoch 22, Batch 1207, Loss: 0.1795
Epoch 22, Batch 1208, Loss: 0.1495
Epoch 22, Batch 1209, Loss: 0.1586
Epoch 22, Batch 1210, Loss: 0.1410
Epoch 22, Batch 1211, Loss: 0.2138
Epoch 22, Batch 1212, Loss: 0.1763
Epoch 22, Batch 1213, Loss: 0.1103
Epoch 22, Batch 1214, Loss: 0.1321
Epoch 22, Batch 1215, Loss: 0.1303
Epoch 22, Batch 1216, Loss: 0.0986
Epoch 22, Batch 1217, Loss: 0.1276
Epoch 22, Batch 1218, Loss: 0.1605
Epoch 22, Batch 1219, Loss: 0.2325
Epoc

In [ ]:
!cp encoder.pth ./drive/MyDrive/SNLP/Extra

In [ ]:
from torch.cuda import empty_cache

transformer_encoder = Encoder(src_vocab_size=trainset.lang.n_words, embedding_layer=nn.Embedding.from_pretrained(embeddings_matrix, freeze=False), n_blocks=2, n_features=embed_size, n_heads=8, n_hidden=512, dropout=0.1, max_length=5000)
encoder_classifier = TransformerClassifier(transformer_encoder, n_features=embed_size)
model = encoder_classifier.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/SNLP/Extra/encoder.pth'))

print("Model loaded correctly \n")

print('Fetching the test data \n')

test_data = pd.read_csv('/content/drive/MyDrive/SNLP/Extra/data/test_2024.csv', quoting = 3)

test_data['label'] = -1

test_df = pd.DataFrame(test_data)
testset = ToxicityDataset('/content/drive/MyDrive/SNLP/Extra/data/test_2024.csv', 'id', 'text', 'label', lang)
test_loader = DataLoader(testset, batch_size=1, shuffle=False, collate_fn=collate)

predictions = pd.DataFrame(columns = ['id', 'label'])
model.eval()

print(f'Predicting labels... \n')

predicted = torch.tensor([]).to(device)
translations = []
for i, data in enumerate(test_loader):
    inputs, mask, labels = data

    inputs = inputs.to(device)
    mask = mask.to(device)

    with torch.no_grad():
      outputs = model(inputs, mask)

      pre = torch.round(torch.sigmoid(outputs)).detach()
      predicted = torch.cat((predicted, pre), dim=0)

    empty_cache()

predicted = predicted.squeeze().cpu().numpy().astype(int)
predictions['label'] = predicted
predictions['id'] = test_data['id']

print(f'Predictions done')

predictions.to_csv('predictions.csv', index = False, header = True)

Model loaded correctly 

Fetching the test data 

Predicting labels... 

Predictions done


In [ ]:
!cp predictions.csv ./drive/MyDrive/SNLP/Extra

In [ ]:
from loaderglove import translateDatasetEntries

In [ ]:
def test_tokenizer():
    glove_file_path = '/content/drive/MyDrive/SNLP/Extra/glove.6B.100d.txt'
    csv_file_path = '/content/drive/MyDrive/SNLP/Extra/testdata.csv'
    glove_embeddings = load_glove_embeddings(glove_file_path)


    lang = Lang("eng", glove_embeddings)


    data = pd.read_csv(csv_file_path)
    sentences = data['text'].values

    for sentence in sentences:
        lang.addSentence(normalizeString(unicodeToAscii(sentence)))

    test_dataset = ToxicityDataset(csv_file_path, id_col='id', text_col='text', label_col='label', lang=lang)

    translated_texts, labels = translateDatasetEntries(test_dataset, lang)

    for original, translated in zip(sentences, translated_texts):
        print(f"Original: {original}")
        print(f"Translated: {translated}")
        print(f"Match: {normalizeString(unicodeToAscii(original)).strip() == translated.strip()}")
        print("-" * 30)

test_tokenizer()

Original: I couldn’t disagree more with this column; Canadians have moved on and don’t care...
Translated: i couldn t disagree more with this column canadians have moved on and don t care . . . EOS
Match: False
------------------------------
Original: I get the odd feeling Klastri the head of the ACLU of...
Translated: i get the odd feeling klastri the head of the aclu of . . . EOS
Match: False
------------------------------
Original: ”Had”... and is now outed as a hypocrite.
Translated:  had . . . and is now outed as a hypocrite . EOS
Match: False
------------------------------
